## 1. 背景前言
在处理数据的过程中，难免会遇到下面类似的场景，自己手里头获得的是简化版的数据字段，但是要比对的或者要合并的却是完整版的数据（有时候也会反过来）

最常见的一个例子就是：在进行地理可视化中，自己收集的数据只保留的缩写，比如北京，广西，新疆，西藏等，但是待匹配的字段数据却是北京市，广西壮族自治区，新疆维吾尔自治区，西藏自治区等，如下。因此就需要有没有一种方式可以很快速便捷的直接进行对应字段的匹配并将结果单独生成一列，就可以用到FuzzyWuzzy库

## 2. FuzzyWuzzy库介绍
FuzzyWuzzy 是一个简单易用的模糊字符串匹配工具包。它依据 Levenshtein Distance 算法，计算两个序列之间的差异。

Levenshtein Distance算法，又叫 Edit Distance算法，是指两个字符串之间，由一个转成另一个所需的最少编辑操作次数。许可的编辑操作包括将一个字符替换成另一个字符，插入一个字符，删除一个字符。一般来说，编辑距离越小，两个串的相似度越大。
### 2.1 安装
这里使用的是Anaconda下的jupyter notebook编程环境，因此在Anaconda的命令行中输入一下指令进行第三方库安装

In [2]:
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple FuzzyWuzzy

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## 2.1 fuzz模块
该模块下主要介绍四个函数（方法），分别为：简单匹配（Ratio）、非完全匹配（Partial Ratio）、忽略顺序匹配（Token Sort Ratio）和去重子集匹配（Token Set Ratio）

注意，注意： 如果直接导入这个模块的话，系统会提示warning，当然这不代表报错，程序依旧可以运行（使用的默认算法，执行速度较慢），可以按照系统的提示安装python-Levenshtein库进行辅助，这有利于提高计算的速度

In [3]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#### 2.1.1 简单匹配（Ratio）
简单的了解一下就行，这个不怎么精确，也不常用

In [5]:
fuzz.ratio("河南省","河南省")
fuzz.ratio("河南","河南省")

80

#### 2.1.2 非完全匹配（Partial Ratio）
尽量使用非完全匹配，精度较高

In [6]:
fuzz.partial_ratio("河南","河南省")

100

#### 2.1.3 忽略顺序匹配（Token Sort Ratio）
原理在于：以 空格 为分隔符，小写 化所有字母，无视空格外的其它标点符号

In [7]:
print(fuzz.ratio("西藏 自治区","自治区 西藏"))
print(fuzz.ratio("I love YOU","YOU LOVE I"))
print(fuzz.token_sort_ratio("西藏 自治区","自治区 西藏"))
print(fuzz.token_sort_ratio("I love YOU","YOU LOVE I"))

50
30
100
100


#### 2.1.4 去重子集匹配（Token Set Ratio）
相当于比对之前有一个集合去重的过程，注意最后两个，可理解为该方法是在token_sort_ratio方法的基础上添加了集合去重的功能，下面三个匹配的都是倒序

In [8]:
print(fuzz.ratio("西藏 西藏 自治区", "自治区 西藏"))
print(fuzz.token_sort_ratio("西藏 西藏 自治区", "自治区 西藏"))
print(fuzz.token_set_ratio("西藏 西藏 自治区", "自治区 西藏"))

40
80
100


fuzz这几个ratio()函数（方法）最后得到的结果都是数字，如果需要获得匹配度最高的字符串结果，还需要依旧自己的数据类型选择不同的函数，然后再进行结果提取，如果但看文本数据的匹配程度使用这种方式是可以量化的，但是对于我们要提取匹配的结果来说就不是很方便了，因此就有了process模块

## 2.2 process模块
用于处理备选答案有限的情况，返回模糊匹配的字符串和相似度。

### 2.2.1 extract提取多条数据
类似于爬虫中select，返回的是列表，其中会包含很多匹配的数据

In [11]:
choices=["河南省","郑州市","湖北省","武汉市"]
process.extract("郑州",choices,limit=2)
# extract之后的数据类型是列表，即使limit=1，最后还是列表，注意和下面extractOne的区别

[('郑州市', 90), ('河南省', 0)]

### 2.2.2 extractOne提取一条数据
如果要提取匹配度最大的结果，可以使用extractOne，注意这里返回的是 元组 类型， 还有就是匹配度最大的结果不一定是我们想要的数据，可以通过下面的示例和两个实战应用体会一下

In [12]:
print(process.extractOne("郑州",choices))
print(process.extractOne("北京",choices))

('郑州市', 90)
('湖北省', 45)
